### 做一个线性回归试试水

In [116]:
import torch
import numpy as np

创造数据集，构造输入x，和对应标签y

In [117]:
x_value = [i for i in range(11)]
print(x_value)
x_train = np.array(x_value, dtype=np.float32)
print(x_train)
x_train = x_train.reshape(-1,1)#把数据转化为矩阵格式
print(x_train)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10.]
[[ 0.]
 [ 1.]
 [ 2.]
 [ 3.]
 [ 4.]
 [ 5.]
 [ 6.]
 [ 7.]
 [ 8.]
 [ 9.]
 [10.]]


In [118]:
y_value = [2*i+1 for i in range(11)]
print(y_value)
y_train = np.array(y_value, dtype=np.float32)#这里格式需要 dtype=np.float32 要不后面报错
print(y_train)
y_train =y_train.reshape(-1,1)
print(y_train)

[1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21]
[ 1.  3.  5.  7.  9. 11. 13. 15. 17. 19. 21.]
[[ 1.]
 [ 3.]
 [ 5.]
 [ 7.]
 [ 9.]
 [11.]
 [13.]
 [15.]
 [17.]
 [19.]
 [21.]]


把数据转化为tensor格式

In [119]:
#torch需要的数据类型为tensor
x_train = torch.from_numpy(x_train)
y_train = torch.from_numpy(y_train)  
print(x_train)
print(y_train)

tensor([[ 0.],
        [ 1.],
        [ 2.],
        [ 3.],
        [ 4.],
        [ 5.],
        [ 6.],
        [ 7.],
        [ 8.],
        [ 9.],
        [10.]])
tensor([[ 1.],
        [ 3.],
        [ 5.],
        [ 7.],
        [ 9.],
        [11.],
        [13.],
        [15.],
        [17.],
        [19.],
        [21.]])


创建线性回归模型

In [120]:
import torch.nn as nn

不管定义多复杂的模型都是一个类

In [121]:
class linear_regression_model(nn.Module):#继承nn.Module类

    def __init__(self, input_dim, output_dim):
        super(linear_regression_model, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)#建立一个全连接层

    def forward(self, x):#前向传播顺序计算
        out = self.linear(x)
        return out
    
input_dim = 1
output_dim = 1

model = linear_regression_model(input_dim,output_dim)#创建一个模型 单层全连接
print(model)

linear_regression_model(
  (linear): Linear(in_features=1, out_features=1, bias=True)
)


指定训练参数

In [122]:
epochs = 1000 #训练1000次
learning_rate = 0.01 #学习率0.01
optimzer = torch.optim.SGD(model.parameters(), lr = learning_rate) #优化器选用SDG（随机梯度下降）优化器，传入模型参数和学习率
criterion = nn.MSELoss()#定义一个损失函数MSE（均方误差）

训练模型

In [123]:
for epoch in range(epochs):
    
    #反向传播之前清零梯度
    optimzer.zero_grad()

    #前向传播
    output = model.forward(x_train)

    #计算损失函数
    loss =  criterion(y_train, output)

    #反向传播
    loss.backward()

    #更新权重
    optimzer.step()
    if epoch %50 == 0:
        print('epoch:{}, loss:{}'.format(epoch, loss.item()))#format 格式化字符串 item()从张量中取出元素

epoch:0, loss:57.30509948730469
epoch:50, loss:0.0645337775349617
epoch:100, loss:0.036807674914598465
epoch:150, loss:0.02099374122917652
epoch:200, loss:0.011974030174314976
epoch:250, loss:0.006829540245234966
epoch:300, loss:0.0038953199982643127
epoch:350, loss:0.0022217349614948034
epoch:400, loss:0.0012672083685174584
epoch:450, loss:0.0007227597525343299
epoch:500, loss:0.00041223366861231625
epoch:550, loss:0.0002351278526475653
epoch:600, loss:0.00013410973770078272
epoch:650, loss:7.64898068155162e-05
epoch:700, loss:4.3627675040625036e-05
epoch:750, loss:2.4883216610760428e-05
epoch:800, loss:1.4192052731232252e-05
epoch:850, loss:8.09635912446538e-06
epoch:900, loss:4.617950253305025e-06
epoch:950, loss:2.6330928903917084e-06


模型预测

In [124]:
#构建测试集
x_test = [i for i in range(11)]
x_test = np.array(x_test, dtype = np.float32)
x_test = x_test.reshape(-1,1)
x_test = torch.from_numpy(x_test)

predict = model.forward(x_test).data.numpy()#预测并把结果转化为 array
predict

array([[ 0.99770707],
       [ 2.9980373 ],
       [ 4.9983673 ],
       [ 6.9986978 ],
       [ 8.999028  ],
       [10.999358  ],
       [12.999689  ],
       [15.000019  ],
       [17.000349  ],
       [19.000679  ],
       [21.001009  ]], dtype=float32)

模型的保存与读取

In [125]:
torch.save(model.state_dict(), "model.pkl")# 把模型权重参数保存（实际为字典），‘model.pkl’

In [126]:
model.load_state_dict(torch.load("model.pkl"))# 加载权重参数

C:\Users\DOCTOR\AppData\Local\Temp\ipykernel_4900\1659317530.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("model.pkl"))# 加载权重参数


<All keys matched successfully>

### 使用GPU训练

只用把数据和模型传入到cuda中就可以了

In [127]:
import torch
import torch.nn as nn
import numpy as np

class linear_regression_model(nn.Module):#继承nn.Module类

    def __init__(self, input_dim, output_dim):
        super(linear_regression_model, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)#建立一个全连接层

    def forward(self, x):#前向传播顺序计算
        out = self.linear(x)
        return out
    
input_dim = 1
output_dim = 1

model = linear_regression_model(input_dim,output_dim)#创建一个模型 单层全连接

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")# 判断gpu是否可用，不可用改为cpu

model.to(device)#把模型传入GPU

epochs = 1000 #训练1000次
learning_rate = 0.01 #学习率0.01
optimzer = torch.optim.SGD(model.parameters(), lr = learning_rate) #优化器选用SDG（随机梯度下降）优化器，传入模型参数和学习率
criterion = nn.MSELoss()#定义一个损失函数MSE（均方误差）

x_train = x_train.to(device)#把数据传入GPU
y_train = y_train.to(device)

for epoch in range(epochs):
    
    #反向传播之前清零梯度
    optimzer.zero_grad()

    #前向传播
    output = model.forward(x_train)

    #计算损失函数
    loss =  criterion(y_train, output)

    #反向传播
    loss.backward()

    #更新权重
    optimzer.step()
    if epoch %50 == 0:
        print('epoch:{}, loss:{}'.format(epoch, loss.item()))#format 格式化字符串 item()从张量中取出元素

epoch:0, loss:36.938751220703125
epoch:50, loss:0.0002704101207200438
epoch:100, loss:0.00015423577860929072
epoch:150, loss:8.797310874797404e-05
epoch:200, loss:5.017521107220091e-05
epoch:250, loss:2.861825851141475e-05
epoch:300, loss:1.6322717783623375e-05
epoch:350, loss:9.309164852311369e-06
epoch:400, loss:5.31049454366439e-06
epoch:450, loss:3.029210347449407e-06
epoch:500, loss:1.7276882999794907e-06
epoch:550, loss:9.85746964943246e-07
epoch:600, loss:5.621758987217618e-07
epoch:650, loss:3.206963299362542e-07
epoch:700, loss:1.8284929126366478e-07
epoch:750, loss:1.0426969510035633e-07
epoch:800, loss:5.9455516066009295e-08
epoch:850, loss:3.3841633495512724e-08
epoch:900, loss:1.932633431067643e-08
epoch:950, loss:1.10174767087301e-08
